# ニューラルネットワークを用いた手書き文字認識

今回の演習では，ニューラルネットワークを利用した手書き文字認識を作成してみます．近年の研究では複雑なニューラルネットワークを大規模データを利用して学習することで，非常に高い識別性能が実現されています．今回の演習では，講義で学習した3層の基本的なニューラルネットワークを利用して，ニューラルネットワークがどのように文字認識を行っているかを確認してみます．

# 0. 今回利用する Python(numpy) の機能

今回の演習で利用する Python の機能について説明します，プログラミングの詳細に興味がない場合は1まで飛ばしてください（ビデオでの説明もありません）

## ベクトルの要素積

ベクトルの要素積は以下のようにベクトル同士を掛けあわせることで計算できます．
今回は行ベクトル $ x = [1, 2, 3, 4, 5]$ を使って計算してみます．

In [ ]:
# 0-1: ベクトルの要素積（アダマール積）

import numpy as np

x = np.array([[1,2,3,4,5]])
print("x * x = ", x * x)

# スカラーとベクトルの計算
ベクトルとスカラーを足し合わせることで，すべての要素に値を足すことが可能です．

In [ ]:
# 0-2: ベクトルとスカラーの足し算

print("x + 1 = ", x + 1)

ベクトルでスカラーを割った場合も，要素ごとの割り算を行うことができます．

In [ ]:
# 0-3: ベクトルとスカラーの割り算

print("5.0 / x =", 5.0/x)

行列，ベクトルの転置は x.T と記述することで，計算可能です．行ベクトルx は x.T により列ベクトルとなります．

In [ ]:
# 0-4: ベクトルの転置

print("x^T = ", x.T)

ベクトル・行列の一般的な積は @ で計算可能です．例えば，xどうしの内積(x,x)は以下で計算可能です．

In [ ]:
# 0-5: 行列・ベクトルの積

print("(x,x) = ", x @ x.T)

exp(x), 平方根などもまとめて計算可能です．

In [ ]:
# 0-6: いろいろな計算

print("exp(x) =", np.exp(x))
print("sqrt(x) = ", np.sqrt(x))

以上を踏まえてニューラルネットの学習を実装してみます．

# 1. 画像のデータの表現
手書き文字認識を実施するにあたり，手書き文字などの画像データがどのように表現されるか考えてみます．まずは，実際に手動（マウス）で文字を書いてみて，それがどのように表現されるか見てみましょう．

In [ ]:
#@title 左の矢印（▷）を押して，下に表示される黒い枠の中にマウスで数字を書いてみてください．書き終わったら finish ボタンを押してください．

from IPython.display import HTML
from io import BytesIO
from PIL import Image as Im
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import matplotlib.pyplot as plt

canvas_html = """
<canvas width=%d height=%d></canvas>
<br>
<button>Finish</button>
<button id="clear">Clear</button>

<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
ctx.fillStyle = 'rgb(0,0,0)';
ctx.fillRect(0, 0, canvas.width, canvas.height);
ctx.strokeStyle = 'white';

var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

 

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

clear.addEventListener("click", function(){
  ctx.fillStyle = 'rgb(0,0,0)';
  ctx.fillRect(0, 0, canvas.width, canvas.height);
});

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

width = 8
height = 8

def draw(filename='drawing.png', w=160, h=160, line_width=30):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  img = Im.open(BytesIO(binary))
  img = img.resize((width,height))
  img = img.convert('L')

  return np.array(img,dtype=np.float)

x = draw()
print("入力した画像は以下のとおりです\n", x)
N = x.shape[1] * x.shape[0]
print("グラフで表すとこうなります（列ベクトルで見づらいため，転置して行ベクトルとして表示しています）\n")
plt.imshow(x, cmap="gray")
plt.show()
print("ベクトルで表現するとこうなります\n")
x = x.reshape([N,1])
print("x^T = ", x.T)


上記の通り，画像は小さなマス目に区切られており，それぞれのマス目（今回は8x8=64個のマス目）に明るさ（輝度）を設定することで構成されています．この明るさを数値により表現し，これを並べてベクトルとすることで解析が行われます．

## 1.1 データの正規化

しかし，画像をそのまま扱った場合，画像を取得した状況に応じてデータのばらつきが生じてしまいます．例としてノートに書かれた数字をカメラで撮影する場合を考えてみます．このような場合，たとえ同じ数字を撮影した場合でも，部屋が明るい場合には全体的に明るい画像（輝度が高い画像）が，暗い場合には暗い画像（輝度が低い画像）が撮影されることになります．このような明るさの影響によるデータのばらつきを抑制するための処理がデータの正規化です．データの正規化には様々な方法がありますが，明るさの平均が0, 分散が1となるように正規化してみます．このような正規化は，以下の式で実現できます．

$ 平均：E(x) = \frac{1}{N}\sum_{j=1}^N x_i = {\bf 1}^T {\bf x} / N$

$ 分散：\sigma^2 = E(x)^2 - E(x^2) = \frac{1}{N}\sum_{j=1}^N x_j^2 - E(x)^2 = {\bf x}^T {\bf x} / N - E(x)^2 $

$ 正規化された画像：{\bf x}' = (x_i - E(x))/\sqrt{\sigma^2}$

ここで，${\bf x} = [x_1, x_2, \cdots, x_N]^T$, $N$はマス目の個数（今回は64）です．

### 問題１：このような正規化操作を実現するための関数を作成してみましょう

In [ ]:
# 1-1: データの正規化

def normalize(x):
  one = np.ones(N) # one はすべての要素が1のベクトルです

  ave = # 問題１−１：平均の計算
  var = # 問題１−２：分散の計算
  
  x_d = # 問題１−３：正規化されたベクトル x_d を計算
  return x_d

# 値がどのように変化するか表示してみます．
print("正規化前")
print(x.T)
print("正規化後")
print(normalize(x).T)

このような正規化を施すことで，文字識別を安定化させることができます．

## 1.2 手書き数字データセットの読み込み

文字認識を行うニューラルネットを構成するためには，手書き文字画像を大量に集める非強うがあります．今回はPython上にあらかじめ準備されたデータセットを利用します．
ここでは下のプログラムを実行してください（変更箇所はありません）．

In [ ]:
# 1-2: 手書き数字データの読み込み
import matplotlib.pyplot as plt

from sklearn import datasets
digits = datasets.load_digits()
labels = digits.target
images = digits.images
# 10 個分のデータを表示する
for i in range(50):
  plt.subplot(5, 10, i+1)
  plt.axis("off")
  plt.title(str(labels[i]))
  plt.imshow(images[i], cmap="gray")

plt.show()
print("画像の総数は", images.shape[0], "です．")


# 2. ニューラルネットの作成と学習
## 2.1 ベクトル・行列による表現

それでは，実際にニューラルネットワークの構成と学習を行っていきます．ニューラルネットの構成は講義資料に則って行いますが，ここではpythonによる実装を簡単にするためにベクトル行列を用いてそれぞれの計算を書き換えます．

$ {\bf u}_i = {\bf w}_0 + W{\bf x}_i $

$ {\bf z}_i = \psi({\bf u})_i$

$ \hat{y}_i = v_0 + {\bf v}^T {\bf z}_i$


ここで，${\bf u}_i$, ${\bf z}_i$，${\bf v}_0$，$W$, $w_0$は以下のとおりです．

${\bf u}_i = [u_{1i}, \cdots, c_{Ni}]^T$

${\bf z}_i = [z_{1i}, \cdots, z_{Ni}]^T$

${\bf v}_i = [v_{1i}, \cdots, v_{Ki}]^T$

$W = 
\left [
\begin{array}{ccc}
w_{11} & \cdots & w_{1N}\\
& \ddots & \\
w_{M1} & \cdots & w{KN}
\end{array}
\right ]
$

$ {\bf w}_0 = [w_{01}, \cdots, w_{0N}]^T$ 

$K$は中間層のノードの数です．

$\hat{y}_i$がニューラルネット全体の出力であり，今回はこの出力が画像に描かれた数字と一致するようにニューラルネットを学習します．画像に描かれた数字が教師データ$y_i$として得られているとし，その誤差$E$を以下のように定義します．

$E = \sum_{i=1}^K (y_i - \hat{y}_i)^2 = \sum_{i=1}^K e_i^2$

講義資料のとおり，ニューラルネットの学習はこの誤差$E$を最小とする$W$, ${\bf w}_0$, ${\bf v}$, $v_0$を求めることに相当します．

まずは，これらの値を入れておくための行列，ベクトルを準備します（ひとまず変更箇所はありません）．

In [ ]:
# 2-1: データの初期化，中間層のノード数を変更する場合は以下を変更して実行

K = 10 # 中間層のノード数

# 入力層と中間層の接続に関するパラメタ
w0 = np.random.rand(K,1)
W = np.random.rand(K, N)

# 中間層と出力層の接続に関するパラメタ
v = np.random.rand(K,1)
v0 = np.random.rand()

これらの行列ベクトルにはひとまずランダムな値を入れておき，これを学習により最適化していきます．

## 2.2 非線形関数

次に，非線形関数としてシグモイド関数を準備します．講義資料にあるとおりシグモイド関数は以下のように定義されています．

$\psi(z) = \frac{1}{1+\exp(-z)}$

### 問題２：シグモイド関数を作成してください．

In [ ]:
# 2-2: 非線形関数（シグモイド関数）

def sigmoid(z):
  psi =　# 問題２：psi(z) の計算結果を psi に代入します．
  return psi

## 2.3 順方向計算の作成

再度の記述となりますが，$y_i$は以下のように計算されます．

$ {\bf u}_i = {\bf w}_0 + W{\bf x}_i $

$ {\bf z}_i = \psi({\bf u})_i$

$ \hat{y}_i = v_0 + {\bf v}^T {\bf z}_i$


### 問題３：それではこれらを使ってまず順伝播（入力${\bf x}_i$から$y_i$を計算するプロセス）を作成しましょう．

In [ ]:
# 2-3: 順方向の計算

def forward_prop(v0, v, w0, W, x_i):
  u_i =   # 問題３−１：入力 x_i から u_i を計算
  z_i =   # 問題３−２：シグモイド関数により z_iを計算
  yh_i =  # 問題３−３：yh_i を計算

  return yh_i, z_i

## 2.4 逆方向計算の作成

次に，評価関数$E$の微分結果を逆方向に反映させていくことで，それぞれの変数に関する微分を計算してみます．

この計算では，$\frac{\partial E}{\partial W}$などを計算する必要がありますが，これらは以下のように$\frac{\partial e_i}{\partial W}$の総和として計算できます．

$ \frac{\partial E}{\partial W} = \sum_{i=1}^n \frac{\partial e_i}{\partial W}$

$ \frac{\partial E}{\partial {\bf v}} = \sum_{i=1}^n \frac{\partial e_i}{\partial {\bf v}}$

そのため，まずは各変数の$e_i$に関する微分を計算します．講義資料のとおり，これらは以下のように計算できます．

$\frac{\partial e_i}{\partial v_0} = -2 (y_i - \hat{y}_i)$

$\frac{\partial e_i}{\partial {v}_k} = -2 (y_i - \hat{y}_i) z_{ik}$

これは，以下のようにまとめることができます．

$
\frac{\partial e_i}{\partial {\bf v}} = 
\left [
\begin{array}{c}
\frac{\partial e_i}{\partial {v}_1} \\ 
\vdots \\
\frac{\partial e_i}{\partial {v}_K}
\end{array}
\right ]
=
\left [
\begin{array}{c}
\frac{\partial e_i}{\partial v_0} z_{i1}\\
\vdots\\
\frac{\partial e_i}{\partial v_0} z_{iK}
\end{array}
\right ]
= \frac{\partial e_i}{\partial v_0} {\bf z}_i
$

また，${w}_{k0}$については以下のように計算できます．

$ \frac{\partial e_i}{\partial w_{k0}} = -2 (y_i - \hat{y}_i) v_k \psi(u_{ik}) (1 - \psi(u_{ik})) = \frac{\partial e_i}{\partial {v}_k}v_k(1-z_{ik})$

これは以下のようにまとめられます．

$
\frac{\partial e_i}{\partial {\bf w}_0} = 
\left [
  \begin{array}{c}
  \frac{\partial e_i}{\partial w_{10}} \\ \vdots \\ \frac{\partial e_i}{\partial w_{K0}}
  \end{array}
\right ]
=
\left [
  \begin{array}{c}
  \frac{\partial e_i}{\partial {v}_1}v_k(1-z_{i1}) \\ \vdots \\
  \frac{\partial e_i}{\partial {v}_K}v_k(1-z_{iK})
  \end{array}
\right ]
 = \frac{\partial e_i}{\partial {\bf v}} \circ {\bf v} \circ ({\bf 1} - {\bf z})
$

ここで，$\circ$はベクトルの要素ごとの積を表します．

同様に${w}_{kj}$についても以下のように計算できます．

$ \frac{\partial e_i}{\partial w_{kj}} = -2 (y_i - \hat{y}_i) v_k \psi(u_{ik}) (1 - \psi(u_{ik}))x_{ij} = \frac{\partial e_i}{\partial w_{k0}}x_{ij}$

これは，以下のようにまとめることができます．

$\frac{\partial e_i}{\partial W} =
\left [
\begin{array}{ccc}
\frac{\partial e_i}{\partial w_{11}} & \cdots & \frac{\partial e_i}{\partial w_{1N}}\\
&  \ddots & \\
\frac{\partial e_i}{\partial w_{K1}} & \cdots & \frac{\partial e_i}{\partial w_{KN}}
\end{array}
\right ]
=
\left [
  \begin{array}{ccc}
  \frac{\partial e_i}{\partial {\bf w}_0} x_{i1} & 
  \cdots & 
  \frac{\partial e_i}{\partial {\bf w}_0} x_{iN} 
  \end{array}
\right ] = 
\frac{\partial e_i}{\partial {\bf w}_0} {\bf x}_i^T
$

以上をまとめると，各変数の微分は以下のように計算できます．

$\frac{\partial e_i}{\partial v_0} = -2 (y_i - \hat{y}_i)\\
\frac{\partial e_i}{\partial {\bf v}} = \frac{\partial e_i}{\partial v_0} {\bf z}_i\\
\frac{\partial e_i}{\partial {\bf w}_0} = \frac{\partial e_i}{\partial {\bf v}} \circ {\bf v} \circ ({\bf 1} - {\bf z})\\
\frac{\partial e_i}{\partial W} =
\frac{\partial e_i}{\partial {\bf w}_0} {\bf x}_i^T
$

### 問題４：それでは逆方向の計算を実装してみましょう．

In [ ]:
# 2-4: 逆方向の計算（誤差逆伝播）

def back_prop(yh_i, y_i, v, z_i, x_i,):
  dy_i =  # 問題４−１：de_i / dy_i 
  dv0_i = # 問題４−２：de_i / dv_0
  dv_i =  # 問題４−３：de_i / dv
  dw0_i = # 問題４−４：de_i / dw0
  dW_i =  # 問題４−５：de_i / dW

  return dv0_i, dv_i, dw0_i, dW_i

## 2.5 $E$に関する微分の計算

これで，$i$番目の画像に対する評価値$e_i$に関する微分を計算することができました．次はこれらの総和を計算することで，全体の評価関数$E$に対する微分を計算してみましょう．

### 問題５：$e_i$の微分の総和から，$E$の微分を計算しましょう．

In [ ]:
# 2-5: 評価関数全体の微分の計算

def derivative_E(images, labels, data_num, v0, v, w0, W):
  # 微分の累積を 0 にセット
  dv0 = 0
  dv = np.zeros([K,1])
  dw0 = np.zeros([K,1])
  dW = np.zeros([K, N])

  for i in range(data_num):
    # 画像をベクトルに変形
    x_i = np.array(images[i],dtype=np.float).reshape(N,1)
    # 正規化
    x_i = normalize(x_i)
    # ラベルを取得
    y_i = labels[i]
    # 順方向計算（先ほど作成した関数により計算）
    yh_i, z_i = forward_prop(v0, v, w0, W, x_i)
    # 逆方向計算（先ほど作成した関数により計算）
    dv0_i, dv_i, dw0_i, dW_i = back_prop(yh_i, y_i, v, z_i, x_i)

    # 問題５：以下を完成させてそれぞれの微分値を計算せよ．
    dv0 = # 問題５−１：dv0_i の総和から dv0を計算
    dv =  # 問題５−２：dv の総和から dv を計算
    dw0 = # 問題５−３：dw0_i の総和から dw0 を計算
    dW =  # 問題５−４：dW_i の総和から dW を計算

  return dv0, dv, dw0, dW

## 2.6 性能チェック機能の作成

これで，ニューラルネットの学習を行うための準備が整いました．これからこれらを利用して繰り返し計算によりニューラルネットワークの学習を行っていきます．ただし，このような学習を行う場合，どこまで学習がすすんだかを確認するため，ニューラルネットの状態をチェックするための機能が必要となります．そこで，現在のパラメタ(${\bf w}_0, W, v_o, {\bf v})と画像データから誤差の総和および現在の正解率（数字が正しく認識される割合）を計算する関数を作成します．

In [ ]:
# 2-6: 性能チェック関数

def neuralnet_exam(images, labels, start, num, v0, v, w0, W):
  # ラベルと出力の2乗誤差の総和
  err = 0
  # 正しく識別あ行われた総数
  correct = 0

  for i in range(start, start + num):
    x_i = normalize(np.array(images[i],dtype=np.float).reshape(N,1))
    yh_i, z_i = forward_prop(v0, v, w0, W, x_i)
    err += (yh_i - labels[i])**2
    if (abs(labels[i] - yh_i) < 0.5):
      correct+=1
  return err, correct

これで，現在の学習状態をチェックすることが可能となりました．

# 3 ニューラルネットの学習とテスト

## 3.1 学習プログラムの作成
それでは，$W$等のパラメタを微分値を用いて更新することで，ニューラルネットの学習を行いましょう．講義資料にあるとおり，これらのパラメタは以下のように更新されます．

$ 
\left [
  \begin{array}{c}
  {v_0^{t+1}} \\ {\bf v}^{t+1} \\ {\bf w}_0^{t+1} \\ W^{t+1}
  \end{array}
\right ]
\left [
  \begin{array}{c}
  {v_0^{t}} \\ {\bf v}^{t} \\ {\bf w}_0^{t} \\ W^{t}
  \end{array}
\right ]
  -
\alpha_t
\left [
  \begin{array}{c}
  \frac{\partial E}{\partial v_0^{t}} |_{(W, {\bf v}) = (W^t, {\bf v}^t)} \\ 
  \frac{\partial E}{\partial {\bf v^{t}}} |_{(W, {\bf v}) = (W^t, {\bf v}^t)}\\ 
  \frac{\partial E}{{\partial \bf w}_0^{t}} |_{(W, {\bf v}) = (W^t, {\bf v}^t)} \\ 
  \frac{\partial E}{\partial W^{t}}|_{(W, {\bf v}) = (W^t, {\bf v}^t)}
  \end{array}
\right ]
$

$\alpha$は更新に応じて変更する方法などもありますが，今回は固定の値として0.25 / （学習に用いるデータの数）を利用します（データ数で割っている理由は，データの数が大きくなると，微分の総和も大きくなるためです）．
### 問題６：下記のプログラムのパラメタ更新部分を作成して実行することで，どのように学習が進むか確認してみましょう（5000回のパラメタ更新を行うため，プログラムの実行には数分かかります，学習終了が表示されるまで待ってください）

In [ ]:
# 3-1: ニューラルネットの学習

# 学習に用いるデータの数
data_num = 1000

# 更新のステップ幅（学習率などとも呼ばれる）
alpha = 0.1 / data_num
# 繰り返し回数，今回は5000回
T = 5000
num = int(T/100)

# 誤差の変化を記録しておくための配列
err_train = np.zeros(num,dtype=np.float)
err_test = np.zeros(num,dtype=np.float)
cor_train = np.zeros(num,dtype=np.float)
cor_test = np.zeros(num,dtype=np.float)

for t in range (T):
  err = 0
  correct = 0
  num = int(T/100)

  # 各パラメタの微分を計算
  dv0, dv, dw0, dW = derivative_E(images, labels, data_num , v0, v, w0, W)

  # 問題６：alpha と微分結果を使って各パラメタを更新しましょう（足し算ではなく引き算になることに注意してください）
  v0 = # 問題６−１：v0をalphaとdv0を使って更新
  v =  # 問題６−２：vをalphaとdvを使って更新
  w0 = # 問題６−３：w0をalphaとdw0を使って更新
  W =  # 問題６−４：WをalphaとdWを使って更新

  # 100回に1回性能をチェック
  if ((t+1) % 100 == 0):
    # 訓練データでチェック
    p = int(t/100)
    err_train[p], cor_train[p] = neuralnet_exam(images, labels, 0, 100, v0, v, w0, W)
    # 検証用データでチェック
    err_test[p], cor_test[p] = neuralnet_exam(images, labels, 1000, 100, v0, v, w0, W)

    print("loop = ", t+1, "error = ",err_train[p], err_test[p],"accuracy = ", cor_train[p], cor_test[p])
print("学習が終了しました")

学習が終了したら，下のプログラムを実行して誤差の遷移をグラフとして表示してみましょう（変更箇所はありません）

In [ ]:
# 3-1-2: 学習遷移の表示

# 誤差の遷移を表示する
plt.figure()
plt.xlabel("epoch")
plt.ylabel("error")
plt.plot(np.arange(50), err_train)
plt.plot(np.arange(50), err_test)

# 認識率の遷移を表示する．
plt.figure()
plt.xlabel("epoch")
plt.ylabel("Accuracy")
plt.ylim(0,100)
plt.plot(np.arange(50), cor_train)
plt.plot(np.arange(50), cor_test)

学習が終わると２つのグラフが表示されたはずです．１つ目のグラフは訓練データ（オレンジ），テストデータ（青）それぞれに対する誤差の遷移を表しており，グラフが下（0）に近づくほど誤差が小さくなっていることを表しています．もうひとつは識別性能の遷移を表しており，上（100%）に近づくほど高い性能を示しています．
グラフを見るとわかりますが，どちらのグラフについても訓練データよりもテストデータの方が悪い値となっていることがわかります．これは，ニューラルネットが訓練データに対して最適化されているため，これに含まれないデータに対しては性能が低下してしまうためです．このような現象は過学習と呼ばれます．これは，学習に用いるデータの数や学習するパラメタの数（中間層の層数やノード数）により変化しますが，一般的に学習データの数を増加させ，また，全体のパラメタ数を減少させることでその影響を軽減させられることがわかっています．

なお，上のプログラム（3-1）を再度実行することで，続きから学習を行うことができます．パラメタが十分に学習されていない場合はこれにより性能を高められることもありますが，パラメタの数（K）が大きい場合には過学習が発生しやすくなります．


## 3.2 パラメタ数の学習への影響

### 問題7：パラメタを変更しての再実行（下の手書き文字認識を実行したあとに行ってください）

それではパラメタの数を変化させてみて，実際に学習結果がどのように変化するか確認してみましょう．中間層のノード数は「2-1:データの初期化」のKの値で設定されていますので，これを異なる数字に変化させて再度学習を行ってみましょう（Kの値を変更したあと，順番に各ブロックを実行してください）．
なお，K=10（初期設定）の場合の結果も提出してもらいますので，再度学習を行う前にグラフと識別結果は撮影しておいてください．


# 4. 手書き文字認識

最後に実際にマウスで文字を書いて手書き文字認識を行ってみましょう

In [ ]:
#@title 左の矢印▷を押して出てくる四角にマウスで数字を書いて認識してみましょう（再実行したい場合は，再度▷を押してください）
canvas_html = """
<canvas width=%d height=%d></canvas>
<br>
<button>Finish</button>
<button id="clear">Clear</button>

<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
ctx.fillStyle = 'rgb(0,0,0)';
ctx.fillRect(0, 0, canvas.width, canvas.height);
ctx.strokeStyle = 'white';

var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

 

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

clear.addEventListener("click", function(){
  ctx.fillStyle = 'rgb(0,0,0)';
  ctx.fillRect(0, 0, canvas.width, canvas.height);
});

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

width = 8
height = 8

def draw2(filename='drawing.png', w=160, h=160, line_width=30):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  img = Im.open(BytesIO(binary))
  img = img.resize((width,height))
  img = img.convert('L')

  return np.array(img,dtype=np.float)

x = draw()
plt.imshow(x, cmap="gray")
plt.show()
y_i, z_i = forward_prop(v0, v, w0, W, normalize(x.reshape(N,1)))
print("識別結果は",y_i,"です")

うまく認識できましたか？今回は非常に基本的な構造を持つニューラルネットワークを利用しているため，そこまで高い識別性能は実現できていません．しかし，現在はより高精度に認識を行うために様々な研究が行われており，非常に高い識別性能を持つニューラルネットワークも実現されています．興味があればそれらについても調べてみると良いでしょう．



お疲れ様でした，以上で今回の演習は終了です．以下の内容を提出してください．
1. K = 10 とした場合の誤差と正解率の遷移のグラフを撮影して提出してください．
2. K の値を 10 より大きい値（例えば5），10より大きい値（例えば20）のそれぞれに変化させ，それぞれの場合の誤差，正解率の遷移の変化を撮影して提出してください．提出する際に，Kの値を下記のレポートに記述してください
3. 2で利用したKの値とそれぞれの場合の結果について，なぜそのような結果になったか考察し，それをレポートに手書きしたものを撮影して提出してください．
